In [1]:
import re
import datetime
import pandas as pd
import json
from osisoft.pidevclub.piwebapi.pi_web_api_client import PIWebApiClient
from pandasgui import show
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import os

In [2]:
client = PIWebApiClient("https://piwebapi.cammesa.com/piwebapi",
                            useKerberos=False, verifySsl=True)

In [5]:
verano1_start = datetime.date(2021, 12, 1)
verano1_end = datetime.date(2022, 3, 1)

verano2_start = datetime.date(2022, 12, 1)
verano2_end = datetime.date(2023, 3, 1)

verano3_start = datetime.date(2023, 12, 1)
verano3_end = datetime.date(2024, 2, 8)

In [6]:
periodos = [
    (verano1_start, verano1_end),
    (verano2_start, verano2_end),
    (verano3_start, verano3_end)
]

In [7]:
df_paths = pd.read_csv('paths.csv')
paths = df_paths['Paths'].tolist()

In [9]:
from osisoft.pidevclub.piwebapi.rest import ApiException
df_nea = pd.DataFrame()

for periodo in periodos:
    start_datetime = datetime.datetime.combine(periodo[0], datetime.time())
    end_datetime = datetime.datetime.combine(periodo[1], datetime.time())

    for k, pt in enumerate(paths):
        try:
            df_interpolate = client.data.get_interpolated_values(
                path=pt,
                start_time=start_datetime,
                end_time=end_datetime,
                interval="1m"
            )

            if df_interpolate.empty or not all(col in df_interpolate.columns for col in ['Timestamp', 'Value']):
                print(f"Tag not found or data is missing for path: {pt}. Skipping...")
                continue 

            df_interpolate = df_interpolate[['Timestamp', 'Value']]
            df_interpolate.set_index('Timestamp', inplace=True)

            column_name = pt.replace('pi:\\\\CAMHIS01\\', '')

            df_interpolate.rename(columns={'Value': column_name}, inplace=True)

            if df_nea.empty:
                df_nea = df_interpolate
            else:
                df_nea = pd.concat([df_nea, df_interpolate], axis=1)

        except ApiException as e:
            if e.status == 404:
                print(f"Tag not found for path: {pt}. Skipping...")
                continue  # Skip to the next iteration
            else:
                # Handle other ApiException or re-raise the exception if needed
                raise




Tag not found for path: pi:\\CAMHIS01\MERCECTE/13/TRAFO3/P.AV. Skipping...
Tag not found for path: pi:\\CAMHIS01\MERCECTE/13/TRAFO3/Q.AV. Skipping...


In [8]:
df_nea.head()

,DEM_NEA/TOT_DEM_____P.AV,RESISTEN/TEMP.AV,RINCON/TEMP.AV,A.BROWN/132/ROCA1/I.AV,A.BROWN/132/ROCA1/P.AV,A.BROWN/132/ROCA1/Q.AV,A.BROWN/132/ROCA2/I.AV,A.BROWN/132/ROCA2/P.AV,A.BROWN/132/ROCA2/Q.AV,ALLEN/132/G.ROCA/I.AV,...,SMARTNEA/132/P.PLAZA/Q.AV,SMARTNEA/132/TRAFO1/I.AV,SMARTNEA/132/TRAFO1/P.AV,SMARTNEA/132/TRAFO1/Q.AV,SMARTNEA/132/TRAFO1/R.AV,SMARTNEA/132/TRAFO1/V.AV,SMARTNEA/132/TRAFO2/I.AV,SMARTNEA/132/TRAFO2/P.AV,SMARTNEA/132/TRAFO2/Q.AV,SMARTNEA/132/TRAFO2/V.AV
Timestamp,,,,,,,,,,,,,,,,,,,,,
2023-12-01T03:00:00Z,1824.61365,26.125362,25.588234,218.200623,41.742195,-25.824749,193.304840,37.545357,-24.484398,116.700000,...,-0.000094,32.369995,6.313385,2.480258,16.0,127.08,78.899994,16.1568,6.7716,127.644000
2023-12-01T03:01:00Z,1809.75700,26.125362,25.588234,219.167450,42.049820,-25.824749,193.143700,37.545357,-24.484398,115.200000,...,-0.000094,32.369995,6.313385,2.480258,16.0,127.08,78.899994,16.1568,6.7716,127.644000
2023-12-01T03:02:00Z,1820.62354,26.125362,25.588234,222.390200,42.027847,-26.044480,193.828552,37.589302,-24.484398,114.500000,...,-0.000094,32.369995,6.313385,2.480258,16.0,127.08,78.899994,16.1568,6.7716,127.644000
2023-12-01T03:03:00Z,1813.32153,26.125362,25.588234,225.693512,41.522470,-25.517128,194.191100,37.589302,-24.484398,112.599991,...,-0.000094,32.369995,6.313385,2.480258,16.0,127.08,78.899994,16.1568,6.7716,128.039978
2023-12-01T03:04:00Z,1805.46729,26.125362,25.588234,240.034790,44.510796,-25.121613,199.065521,38.050736,-23.935074,114.000000,...,-0.000094,32.369995,6.313385,2.480258,16.0,127.08,78.899994,16.1568,6.7716,128.039978


In [9]:
df_nea_hora = df_nea.copy()
df_nea_hora.index = pd.to_datetime(df_nea_hora.index).tz_convert('America/Argentina/Buenos_Aires').tz_localize(None)

df_nea_hora

,DEM_NEA/TOT_DEM_____P.AV,RESISTEN/TEMP.AV,RINCON/TEMP.AV,A.BROWN/132/ROCA1/I.AV,A.BROWN/132/ROCA1/P.AV,A.BROWN/132/ROCA1/Q.AV,A.BROWN/132/ROCA2/I.AV,A.BROWN/132/ROCA2/P.AV,A.BROWN/132/ROCA2/Q.AV,ALLEN/132/G.ROCA/I.AV,...,SMARTNEA/132/P.PLAZA/Q.AV,SMARTNEA/132/TRAFO1/I.AV,SMARTNEA/132/TRAFO1/P.AV,SMARTNEA/132/TRAFO1/Q.AV,SMARTNEA/132/TRAFO1/R.AV,SMARTNEA/132/TRAFO1/V.AV,SMARTNEA/132/TRAFO2/I.AV,SMARTNEA/132/TRAFO2/P.AV,SMARTNEA/132/TRAFO2/Q.AV,SMARTNEA/132/TRAFO2/V.AV
Timestamp,,,,,,,,,,,,,,,,,,,,,
2023-12-01 00:00:00,1824.61365,26.125362,25.588234,218.200623,41.742195,-25.824749,193.304840,37.545357,-24.484398,116.700000,...,-0.000094,32.369995,6.313385,2.480258,16.0,127.080000,78.899994,16.1568,6.7716,127.644000
2023-12-01 00:01:00,1809.75700,26.125362,25.588234,219.167450,42.049820,-25.824749,193.143700,37.545357,-24.484398,115.200000,...,-0.000094,32.369995,6.313385,2.480258,16.0,127.080000,78.899994,16.1568,6.7716,127.644000
2023-12-01 00:02:00,1820.62354,26.125362,25.588234,222.390200,42.027847,-26.044480,193.828552,37.589302,-24.484398,114.500000,...,-0.000094,32.369995,6.313385,2.480258,16.0,127.080000,78.899994,16.1568,6.7716,127.644000
2023-12-01 00:03:00,1813.32153,26.125362,25.588234,225.693512,41.522470,-25.517128,194.191100,37.589302,-24.484398,112.599991,...,-0.000094,32.369995,6.313385,2.480258,16.0,127.080000,78.899994,16.1568,6.7716,128.039978
2023-12-01 00:04:00,1805.46729,26.125362,25.588234,240.034790,44.510796,-25.121613,199.065521,38.050736,-23.935074,114.000000,...,-0.000094,32.369995,6.313385,2.480258,16.0,127.080000,78.899994,16.1568,6.7716,128.039978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-07 23:56:00,2327.74658,28.176205,30.666515,316.736328,70.702590,-20.924774,327.492249,73.009766,-27.780346,102.300000,...,-0.000094,40.004997,7.722623,2.705737,20.0,121.440000,96.899994,18.9486,7.6626,122.099991
2024-02-07 23:57:00,2326.25500,28.176205,30.666515,316.011230,70.702590,-20.924774,299.776600,69.252380,-28.263752,102.300000,...,-0.000094,40.710000,7.722623,2.818476,20.0,121.679993,96.899994,18.9486,7.6626,122.099991
2024-02-07 23:58:00,2330.41968,28.176205,30.520023,316.011230,70.702590,-20.924774,290.994568,67.955980,-28.373615,102.800000,...,-0.000094,40.710000,7.722623,2.818476,20.0,121.679993,96.899994,18.9486,7.6626,122.099991


In [10]:
df_nea.to_csv('colapsos_verano3_features.csv', index=True)